In [9]:
import nltk
import spacy

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from tabulate import tabulate

In [10]:
nlp = spacy.load("en_core_web_sm")
# Get English stopwords
stop_words = set(stopwords.words('english'))

def process_text_corpus(text_corpus):
    # Tokenize the text into words
    words = word_tokenize(text_corpus.lower())

    # Remove stopwords and punctuation
    # filtered_words = [word for word in words if word not in stop_words and word not in punctuation]
    print(words)
# A paradigm class refers to a group of lexemes or words that share similar inflectional patterns and belong to the same grammatical category. Lexemes within the same paradigm class exhibit similar variations based on grammatical features such as tense, number, case, person, and so on.
sample_text = input("Enter a text: ")
table_data = process_text_corpus(sample_text)



['a', 'paradigm', 'class', 'refers', 'to', 'a', 'group', 'of', 'lexemes', 'or', 'words', 'that', 'share', 'similar', 'inflectional', 'patterns', 'and', 'belong', 'to', 'the', 'same', 'grammatical', 'category', '.', 'lexemes', 'within', 'the', 'same', 'paradigm', 'class', 'exhibit', 'similar', 'variations', 'based', 'on', 'grammatical', 'features', 'such', 'as', 'tense', ',', 'number', ',', 'case', ',', 'person', ',', 'and', 'so', 'on', '.']


In [ ]:
import re

def generate_bigrams_with_eos(tokens):
    
    # Add 'eos' at the beginning and end of the list
    tokens = ['eos'] + tokens
    
    # Replace end-of-sentence punctuation with 'eos'
    tokens = [re.sub(r'[.!?]$', 'eos', token) for token in tokens]
    
    # Remove remaining punctuation
    tokens = [re.sub(r'[^\w\s]', '', token) for token in tokens]
    
    bigram_list = []
    for i in range(len(tokens) - 1):
        bigram_list.append((tokens[i], tokens[i + 1]))
    return bigram_list

# Sample text data
text = "This is a sample text. You can replace it with your own text. What do you think? It's great!"

# Tokenize the text into words
words = nltk.word_tokenize(text)

# Generate bigrams with 'eos' using the custom function
bi_grams = generate_bigrams_with_eos(words)

# Calculate frequency distribution for bigrams
freq_dist = FreqDist(bi_grams)

# Calculate conditional frequency distribution for bigrams
cond_freq_dist = ConditionalFreqDist(bi_grams)

# Create a probability table for bigrams
prob_table = {}
for bigram in freq_dist:
    # Calculate the probability of each bigram
    prob = freq_dist[bigram] / cond_freq_dist[bigram[0]].N()
    prob_table[bigram] = prob

# Print the probability table
for bigram, prob in prob_table.items():
    print(f"{bigram}: {prob}")


In [29]:
import re
from collections import defaultdict
from tabulate import tabulate

def preprocess(text):
    text = nltk.word_tokenize(text.lower())
    text = ['eos'] + text
    text = [re.sub(r'[.!?]$', 'eos', token) for token in text]
    text = [re.sub(r'[^\w\s]', '', token) for token in text]
    return text

def create_bigrams(data):
    bigrams = []
    bigram_counts = {}
    unigram_counts = {}
    for i in range(len(data) - 1):
        if i < len(data) - 1 and data[i + 1].islower():
            bigrams.append((data[i], data[i + 1]))

            if (data[i], data[i + 1]) in bigram_counts:
                bigram_counts[(data[i], data[i + 1])] += 1
            else:
                bigram_counts[(data[i], data[i + 1])] = 1

        if data[i] in unigram_counts:
            unigram_counts[data[i]] += 1
        else:
            unigram_counts[data[i]] = 1
    return bigrams, unigram_counts, bigram_counts

def calc_bigram_prob(bigrams, unigram_counts, bigram_counts, vocab_size):
    prob_dict = {}
    for bigram in bigrams:
        word1, word2 = bigram
        prob_dict[bigram] = (bigram_counts.get(bigram, 0) + 1) / (unigram_counts.get(word1, 0) + vocab_size)
    return prob_dict

def gen_bigrams(tokens):
    return [(tokens[i], tokens[i + 1]) for i in range(len(tokens) - 1)]

def calc_unigram_freq(tokens):
    unigram_freq = defaultdict(int)
    for word in tokens:
        unigram_freq[word] += 1
    return unigram_freq

def calc_bigram_freq(bigrams):
    bigram_freq = defaultdict(int)
    for bigram in bigrams:
        bigram_freq[bigram] += 1
    return bigram_freq

def calc_bigram_probs(bigram_freq, unigram_freq, vocab_size):
    bigram_probs = defaultdict(float)
    for bigram in bigram_freq:
        bigram_probs[bigram] = bigram_freq[bigram] / unigram_freq[bigram[0]]
    return bigram_probs

def bi_matrix(bigrams, bigram_probs):
    unique_words = list(set([word for bigram in bigrams for word in bigram]))
    unique_words.sort()

    matrix_data = []
    for word1 in unique_words:
        row = [word1] + [f"{bigram_probs.get((word1, word2), 0):.4f}" for word2 in unique_words]
        matrix_data.append(row)

    headers = [''] + unique_words
    table = tabulate(matrix_data, headers, tablefmt="grid")
    print("\nBigram Probability Matrix:")
    print(table)

def display_bi_freq(bigram_freq):
    table_data = [(bigram, freq) for bigram, freq in bigram_freq.items()]
    print("\nBigram Frequency:")
    print(tabulate(table_data, headers=["Bigram", "Frequency"], tablefmt="grid"))

def display_uni_freq(unigram_freq):
    table_data = [(unigram, freq) for unigram, freq in unigram_freq.items()]
    print("\nUnigrams Frequency:")
    print(tabulate(table_data, headers=["Unigram", "Frequency"], tablefmt="grid"))

def bigram_list_ip(sentence_bigrams):
    print("\nBigrams in input sentence:")
    print(sentence_bigrams)

def calc_sentence_prob(sentence, bigram_probs):
    sentence_bigrams = gen_bigrams(sentence)
    prob = 1.0
    for bigram in sentence_bigrams:
        prob *= bigram_probs.get(bigram, 0)
    return prob

if __name__ == '__main__':
    corpus = input("Enter training corpus")
    # corpus = "(eos) You book a flight (eos) I read a book (eos) You read (eos)"
    preproc_corpus = preprocess(corpus)
    tokens = preproc_corpus

    bigrams, unigram_counts, bigram_counts = create_bigrams(tokens)

    unigram_freq = calc_unigram_freq(tokens)
    bigram_freq = calc_bigram_freq(bigrams)

    vocab_size = len(set(tokens))
    bigram_probs = calc_bigram_prob(bigrams, unigram_counts, bigram_counts, vocab_size)

    display_uni_freq(unigram_freq)
    display_bi_freq(bigram_freq)

    bi_matrix(bigrams, bigram_probs)

    ip_sentence = input("Enter test sentence")
    # ip_sentence = "(eos) You read a book (eos)"
    ip_tokens = preprocess(ip_sentence)

    bigram_list_ip(gen_bigrams(ip_tokens))

    ip_prob = calc_sentence_prob(ip_tokens, bigram_probs)
    print(f"\nProbability of given sentence: {ip_prob:.6f}")

    bigram_probs_smoothed = calc_bigram_prob(bigrams, unigram_counts, bigram_counts, vocab_size + 1)

    print("\nAfter Add-One Smoothing:")
    bi_matrix(bigrams, bigram_probs_smoothed)

    ip_prob_smoothed = calc_sentence_prob(ip_tokens, bigram_probs_smoothed)
    print(f"\nProbability of given sentence (After Add-One Smoothing): {ip_prob_smoothed:.6f}")



Unigrams Frequency:
+-----------+-------------+
| Unigram   |   Frequency |
+===========+=============+
| eos       |           4 |
+-----------+-------------+
| you       |           2 |
+-----------+-------------+
| book      |           2 |
+-----------+-------------+
| a         |           2 |
+-----------+-------------+
| flight    |           1 |
+-----------+-------------+
| i         |           1 |
+-----------+-------------+
| read      |           2 |
+-----------+-------------+

Bigram Frequency:
+-------------------+-------------+
| Bigram            |   Frequency |
+===================+=============+
| ('eos', 'you')    |           2 |
+-------------------+-------------+
| ('you', 'book')   |           1 |
+-------------------+-------------+
| ('book', 'a')     |           1 |
+-------------------+-------------+
| ('a', 'flight')   |           1 |
+-------------------+-------------+
| ('flight', 'eos') |           1 |
+-------------------+-------------+
| ('eos', 'i')  